In [1]:
import os

import pandas as pd

In [2]:
# general configs
pd.set_option("display.max_columns", 500)

# condiciones - analysis

In [3]:
# setting configs and constants
# path to raw dataset
PATH = "../../../data/interim/1. col_red"
FILENAME = "VIGABT_POLIZAS-red_col.feather"
RAW_FILE = os.path.join(PATH, FILENAME)

if not(os.path.exists(RAW_FILE) and os.path.isfile(RAW_FILE)):
    raise Exception("File doesn't exists.")

## Loading database

In [4]:
# usecols = []
df = pd.read_feather(RAW_FILE)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2363937 entries, 0 to 2363936
Data columns (total 28 columns):
ANTIG                           float64
CANT_RENOVACION                 float64
CAPITAL_ACCESORIOS              object
CAPITAL_ASEGURADO_COTIZACION    object
CAPITAL_VEHICULO                object
COD_COBRO                       object
COD_COBRO_ANTERIOR              object
COD_POSTAL                      int64
COD_PROD                        int64
COD_RAMO                        int64
COD_RIES                        int64
COD_ZONA_CASCO                  float64
COD_ZONA_RC                     float64
COD_ZONA_ROBO                   float64
CONV_COMISIONARIO               object
FECHA_PROCESO                   datetime64[ns]
FECHA_VENC_POL                  datetime64[ns]
FECHA_VIG_ORIG_POL              datetime64[ns]
FECHA_VIG_POL                   datetime64[ns]
ID                              float64
MCA_AGRAVANTE                   object
MCA_EMPLEADO                    ob

In [6]:
df["ID"].count()

596179

In [7]:
df["ID"].isna().mean()

0.7478025006588586

In [18]:
df["NUM_SECU_POL"].value_counts(dropna=False).head()

1603664279999    43343
1601618549999    38173
1789488999999    26924
1855096939999    23366
1723405719999    22320
Name: NUM_SECU_POL, dtype: int64

In [19]:
df[df["NUM_SECU_POL"] == 1603664279999]["FECHA_VENC_POL"].value_counts(dropna=False)

2017-12-31    43343
Name: FECHA_VENC_POL, dtype: int64

In [11]:
# df["COD_RIES"].value_counts(dropna=False)

## formatting columns

## dropping useless columns

In [20]:
df.drop(columns=["ANTIG"], inplace=True) # "COD_RIES"

## deleting rows

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2363937 entries, 0 to 2363936
Data columns (total 27 columns):
CANT_RENOVACION                 float64
CAPITAL_ACCESORIOS              object
CAPITAL_ASEGURADO_COTIZACION    object
CAPITAL_VEHICULO                object
COD_COBRO                       object
COD_COBRO_ANTERIOR              object
COD_POSTAL                      int64
COD_PROD                        int64
COD_RAMO                        int64
COD_RIES                        int64
COD_ZONA_CASCO                  float64
COD_ZONA_RC                     float64
COD_ZONA_ROBO                   float64
CONV_COMISIONARIO               object
FECHA_PROCESO                   datetime64[ns]
FECHA_VENC_POL                  datetime64[ns]
FECHA_VIG_ORIG_POL              datetime64[ns]
FECHA_VIG_POL                   datetime64[ns]
ID                              float64
MCA_AGRAVANTE                   object
MCA_EMPLEADO                    object
MCA_MOVIMIENTO                  obj

In [14]:
# removing duplicated rows
# print(df.duplicated().sum())
# df.drop_duplicates(inplace=True) # getting first element

158182


### getting unique NUM_SECU_POL

In [21]:
df.drop_duplicates(subset=["NUM_SECU_POL"], keep="last", inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 729756 entries, 0 to 2363936
Data columns (total 27 columns):
CANT_RENOVACION                 709911 non-null float64
CAPITAL_ACCESORIOS              729756 non-null object
CAPITAL_ASEGURADO_COTIZACION    725415 non-null object
CAPITAL_VEHICULO                729756 non-null object
COD_COBRO                       729756 non-null object
COD_COBRO_ANTERIOR              604173 non-null object
COD_POSTAL                      729756 non-null int64
COD_PROD                        729756 non-null int64
COD_RAMO                        729756 non-null int64
COD_RIES                        729756 non-null int64
COD_ZONA_CASCO                  729754 non-null float64
COD_ZONA_RC                     729754 non-null float64
COD_ZONA_ROBO                   729754 non-null float64
CONV_COMISIONARIO               729756 non-null object
FECHA_PROCESO                   729756 non-null datetime64[ns]
FECHA_VENC_POL                  729756 non-null datetim

In [22]:
df["NUM_SECU_POL"].value_counts(dropna=False).head()

1681059659999    1
1656081689999    1
1829065239999    1
1699287149999    1
1827522029999    1
Name: NUM_SECU_POL, dtype: int64

In [23]:
df["cambio_cobro"] = (df["COD_COBRO"] != df["COD_COBRO_ANTERIOR"]) & (df["COD_COBRO_ANTERIOR"].notna())

In [24]:
df["ANTIG_calc"] = (df["FECHA_VIG_POL"] - df["FECHA_VIG_ORIG_POL"]).dt.days

# saving into feather
convert format to save into feather


In [25]:
file_save = FILENAME.split(".")[0]
file_save

'VIGABT_POLIZAS-red_col'

In [26]:
to_save = f"../../../data/interim/3. row_red/{file_save}-red_rows-clean.feather"
df.reset_index(drop=True).to_feather(to_save)